In [0]:
!pip install tensorflow==1.14.0

## Introduction
* For this notebook we will be exploring other stuctures to classify proteins from their sequence
* We will explore 4 different ways
* The first way, we will try changing some of the architecture using the emebedding and CONV1d
* The second way, we ill trying implemeting an LSTM after the embedding layer
* The third way, we will try implementing a GRU after the embedding layer
* The fourth way, we will try implementing bi bi-diertional LSTMS

## First Load in the data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:
proteins = pd.read_csv("/content/drive/My Drive/UNH Spring/Protein_seq_project/top_15_proteins.csv")

In [0]:
proteins.head()

,Unnamed: 0,sequence,classification,num_residues
0,67,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE,286
1,68,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE,286
2,74,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,LIGASE,330
3,75,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,LIGASE,330
4,76,KESAAAKFERQHMDSGNSPSSSSNYCNLMMCCRKMTQGKCKPVNTF...,HYDROLASE,124


In [0]:
from sklearn.preprocessing import LabelBinarizer

# Transform labels to one-hot
lb = LabelBinarizer()
Y = lb.fit_transform(proteins['classification'])

In [0]:
lb.classes_,len(lb.classes_)

(array(['HYDROLASE', 'HYDROLASE/HYDROLASE INHIBITOR', 'IMMUNE SYSTEM',
        'ISOMERASE', 'LIGASE', 'LYASE', 'OXIDOREDUCTASE', 'RIBOSOME',
        'RIBOSOME/ANTIBIOTIC', 'SIGNALING PROTEIN', 'TRANSCRIPTION',
        'TRANSFERASE', 'TRANSPORT PROTEIN', 'VIRAL PROTEIN', 'VIRUS'],
       dtype='<U29'), 15)

In [0]:
Y[5]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Vectorize the Sequences
* This time try adding padding pre

In [0]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# maximum length of sequence, everything afterwards is discarded!
max_length = 512

#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(proteins['sequence'])
#represent input data as word rank number sequences
X = tokenizer.texts_to_sequences(proteins['sequence'])
X = sequence.pad_sequences(X, maxlen=max_length,padding='pre')

In [0]:
tokenizer.word_index

{'a': 1,
 'b': 24,
 'c': 17,
 'd': 11,
 'e': 5,
 'f': 14,
 'g': 2,
 'h': 18,
 'i': 9,
 'k': 7,
 'l': 3,
 'm': 19,
 'n': 13,
 'o': 25,
 'p': 12,
 'q': 15,
 'r': 10,
 's': 6,
 't': 8,
 'u': 20,
 'v': 4,
 'w': 21,
 'x': 22,
 'y': 16,
 'z': 23}

### Split our data

In [0]:
X.shape,Y.shape

((283101, 512), (283101, 15))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.1)

In [0]:
#keep X_test and y_test until the end get valids
X_test.shape
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.1)

## Second Model
* This notebook is a continuation from another one because I ran out of memory on my computer
* Use and LSTM layer after the embedding layer

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 16
depth  = 20

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(LSTM(depth))
model.add(Dense(15, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 512, 16)           416       
_________________________________________________________________
lstm_4 (LSTM)                (None, 20)                2960      
_________________________________________________________________
dense_4 (Dense)              (None, 15)                315       
Total params: 3,691
Trainable params: 3,691
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.layers[0].output, model.input

(<tf.Tensor 'embedding_4/embedding_lookup/Identity:0' shape=(?, 512, 16) dtype=float32>,
 <tf.Tensor 'embedding_4_input:0' shape=(?, 512) dtype=float32>)

In [0]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid), 
          epochs=50, 
          batch_size=128)

Train on 229311 samples, validate on 25479 samples
Epoch 1/50
229311/229311 [==============================] - 675s 3ms/step - loss: 2.0395 - acc: 0.3361 - val_loss: 1.8569 - val_acc: 0.3861
Epoch 2/50
229311/229311 [==============================] - 677s 3ms/step - loss: 1.8169 - acc: 0.3988 - val_loss: 1.7074 - val_acc: 0.4263
Epoch 3/50
229311/229311 [==============================] - 686s 3ms/step - loss: 1.6999 - acc: 0.4314 - val_loss: 1.6545 - val_acc: 0.4445
Epoch 4/50
229311/229311 [==============================] - 684s 3ms/step - loss: 1.6162 - acc: 0.4604 - val_loss: 1.5779 - val_acc: 0.4716
Epoch 5/50
229311/229311 [==============================] - 689s 3ms/step - loss: 1.5684 - acc: 0.4792 - val_loss: 1.5461 - val_acc: 0.4879
Epoch 6/50
229311/229311 [==============================] - 688s 3ms/step - loss: 1.5289 - acc: 0.4907 - val_loss: 1.5244 - val_acc: 0.4950
Epoch 7/50
229311/229311 [==============================] - 689s 3ms/step - loss: 1.5153 - acc: 0.4952 - val_